In [1]:
import pandas as pd

In [2]:
missing_fields = pd.read_csv('data/Sotheby\'s Missing Required FIelds - Sotheby Missing Fields.csv')
missing_fields.head()

,Brokerage,Listing ID,Address,Unit,Agents,Error messages
0,"Sotheby's International Realty, Inc.",7751685,35 Hudson Yards,7003,"Matthew Perceval, Daniel Chang",CoListAgent Office of associated Members must ...
1,"Sotheby's International Realty, Inc.",7751131,425 East 63rd Street,E8J,Deborah Forest,ListAgent Office of associated Members must ma...
2,"Sotheby's International Realty, Inc.",7747729,460 West 51st Street,5,Marcelle Cohen,AvailabilityDate is required for rentals
3,"Sotheby's International Realty, Inc.",7747728,460 West 51st Street,4,Marcelle Cohen,AvailabilityDate is required for rentals
4,"Sotheby's International Realty, Inc.",7739566,257 West 17th Street,PHB,Kristi Ambrosetti,PropertyCondition is required when PropertyTyp...


In [5]:
error = "CoListAgent Office of associated Members must, CoExclusiveAgentKey is required when ListingAgreement is CoExclusive, CoExclusiveOfficeKey is required when ListingAgreement is CoExclusive, FireplaceFeatures is required if FireplaceYN is True or FireplacesTotal is greater than 0"

In [8]:
error.split(', ')

['CoListAgent Office of associated Members must',
 'CoExclusiveAgentKey is required when ListingAgreement is CoExclusive',
 'CoExclusiveOfficeKey is required when ListingAgreement is CoExclusive',
 'FireplaceFeatures is required if FireplaceYN is True or FireplacesTotal is greater than 0']

In [9]:
error_list = error.split(', ')
flist = []
for e in error_list:
    tmp_lst = e.split()[:2]
    if tmp_lst[1][0].isupper():
        flist.append(" ".join(tmp_lst))
    else:
        flist.append(tmp_lst[0])
flist

['CoListAgent Office',
 'CoExclusiveAgentKey',
 'CoExclusiveOfficeKey',
 'FireplaceFeatures']

In [10]:
def create_key(error):
    error_list = error.split(', ')
    flist = []
    for e in error_list:
        tmp_lst = e.split()[:2]
        if tmp_lst[1][0].isupper():
            flist.append(" ".join(tmp_lst))
        else:
            flist.append(tmp_lst[0])
    return '|'.join(flist)

In [12]:
missing_fields['error_key'] = [create_key(x) for x in missing_fields['Error messages']]

In [13]:
all_keys = []
for k in missing_fields['error_key']:
    all_keys += (k.split('|'))
all_keys = set(all_keys)

In [14]:
stats = {'missing_field': [], 'num_listings': []}
for k in all_keys:
    count = 0
    for i in missing_fields['error_key']:
        if k in i:
            count += 1
    stats['missing_field'].append(k)
    stats['num_listings'].append(count)

In [18]:
stats = pd.DataFrame(stats)
stats

,missing_field,num_listings
0,PropertyCondition,33
1,BuildingAreaUnits,43
2,BuyerOfficeName,1
3,FurnishedListPrice,1
4,AttendanceType,26
5,BuyerAgentEmail,1
6,TaxLot,40
7,LotSizeUnits,50
8,CoListAgent Office,1
9,ListingAgreement,49


In [17]:
def report_writer(dfs:dict, report_name:str):
    """Creates an excel report using a dictionary of dfs.

    Args:
        dfs (dict): Data
        report_name (str): Name of file.
    """
    writer = pd.ExcelWriter(report_name, engine='xlsxwriter')
    workbook = writer.book
    format = workbook.add_format()
    format.set_align('center')
    for sheetname, df in dfs.items():
        df.to_excel(writer, sheet_name=sheetname, index=False)
        worksheet = writer.sheets[sheetname]
        for idx, col in enumerate(df):
            series =df[col]
            max_len = max((
                series.astype(str).map(len).max(),
                len(str(series.name))
            ))+1
            worksheet.set_column(idx, idx, max_len, format)

    writer.close()

In [19]:
report_writer({'errors_with_keys': missing_fields, 'summary_stats': stats}, 'data/sothebys_errors_with_summary.xlsx')